<a href="https://colab.research.google.com/github/pinggu95/deep_dive_AI/blob/main/57%EC%9D%BC%EC%B0%A8/57%EC%9D%BC%EC%B0%A8_CTM_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 복합 토픽 모델링(Combined Topic Modeling)


# Contextualized Topic Models, CTM 설치

contextualized topic model 라이브러리를 설치합시다.

In [ ]:
%%capture
!pip install contextualized-topic-models

In [ ]:
%%capture
!pip install pyldavis

## 노트북 재시작 필요

In [ ]:
!pip install Gensim

# 데이터 로드

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

In [ ]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [ ]:
!head -n 3 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she


In [ ]:
!head -n 5 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and
Henry Howard, 13th Earl of Suffolk, 6th Earl of Berkshire (8 August 1779 – 10 August 1779) was a British peer, the son of Henry Howard, 12th Earl of Suffolk. His father died on 7 March 1779, leaving behind his pregnant widow. The Earldom of Suffolk became dormant until she
Marinko Matošević (Croatian pronunciation: [mariŋko matoʃeʋit͡ɕ]; born 8 August 1985) is an Aus

In [ ]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt" # EDIT THIS WITH THE FILE YOU UPLOAD

# 임포트

In [ ]:
!pip install numpy

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk

## 전처리

In [ ]:
from nltk.corpus import stopwords as stop_words

nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:2000]]

stopwords = list(stop_words.words("english"))

sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipython-input-2846991197.py:5: ResourceWarning: unclosed file <_io.TextIOWrapper name='dbpedia_sample_abstract_20k_unprep.txt' mode='r' encoding='utf-8'>
  documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:2000]]


In [ ]:
# normalization 전처리 후 문서
preprocessed_documents[:2]

['mid peninsula highway across peninsula canadian province ontario although highway connecting fort south decades international study published ministry',
 'died march american photographer photography operated studio silver spring maryland later lived florida magazine photographer year']

In [ ]:
# 전처리 전 문서 == documnets와 동일
unpreprocessed_corpus[:2]

['The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry',
 "Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and"]

In [ ]:
# 전체 단어 집합의 개수
print('bag of words에 사용 될 단어 집합의 개수 :',len(vocab))

bag of words에 사용 될 단어 집합의 개수 : 2000


In [ ]:
vocab[:5]

['illinois', 'colorado', 'russia', 'members', 'indonesia']

In [ ]:
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
tp.vocab[:10]

array(['ab', 'abbreviated', 'abroad', 'academic', 'academy', 'accepted',
       'access', 'according', 'accounting', 'achieved'], dtype=object)

In [ ]:
len(tp.vocab)

2000

단어 집합의 상위 10개 단어를 출력해봅시다. 여기서 출력하는 tp.vocab과 앞에서의 vocab은 집합 관점에서는 같습니다.

In [ ]:
set(vocab) == set(tp.vocab)

True

## Combined TM 학습하기


In [ ]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=20, num_epochs=10)
ctm.fit(training_dataset) # run the model

Epoch: [10/10]	 Seen Samples: [19840/20000]	Train Loss: 137.8072312878024	Time: 0:00:00.693458: : 10it [00:06,  1.46it/s]
100%|██████████| 32/32 [00:00<00:00, 61.61it/s]


In [ ]:
ctm.get_topic_lists(5)

[['lies', 'km', 'village', 'mi', 'capital'],
 ['county', 'states', 'community', 'state', 'unincorporated'],
 ['played', 'former', 'football', 'competed', 'born'],
 ['born', 'politician', 'party', 'player', 'served'],
 ['used', 'number', 'work', 'designed', 'large'],
 ['group', 'founded', 'published', 'books', 'book'],
 ['american', 'best', 'played', 'film', 'made'],
 ['published', 'list', 'modern', 'capital', 'islands'],
 ['released', 'album', 'studio', 'game', 'band'],
 ['family', 'species', 'politician', 'genus', 'served'],
 ['station', 'road', 'railway', 'island', 'line'],
 ['politician', 'member', 'party', 'died', 'university'],
 ['film', 'released', 'directed', 'produced', 'album'],
 ['family', 'found', 'species', 'moth', 'mm'],
 ['house', 'building', 'built', 'school', 'th'],
 ['league', 'championship', 'football', 'final', 'season'],
 ['south', 'mi', 'km', 'kilometres', 'administrative'],
 ['william', 'based', 'work', 'public', 'author'],
 ['population', 'built', 'located', 'tow

# 시각화


In [ ]:
 lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

100%|██████████| 32/32 [00:00<00:00, 64.41it/s]


In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

100%|██████████| 32/32 [00:00<00:00, 41.87it/s]


In [ ]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

100%|██████████| 32/32 [00:01<00:00, 30.75it/s]


In [ ]:
# 전처리 문서의 첫번째 문서
print(preprocessed_documents[0])

mid peninsula highway across peninsula canadian province ontario although highway connecting fort south decades international study published ministry


In [ ]:
import numpy as np
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document

In [ ]:
ctm.get_topic_lists(5)[topic_number] #and the topic should be about natural location related things

['population', 'built', 'located', 'town', 'city']